In [26]:
import sys
IN_COLAB = 'google.colab' in sys.modules
# Reference: https://colab.research.google.com/github/pytorch/xla/blob/master/contrib/colab/pytorch-xla-profiling-colab.ipynb#scrollTo=Micd3xZvoA-c
if IN_COLAB:
    %pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

In [ ]:
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [27]:
import torch
import torch.nn as nn
from torch.nn import functional as F

if IN_COLAB:
    # imports the torch_xla package
    import torch_xla
    import torch_xla.core.xla_model as xm

torch.manual_seed(1337) ## To get repeatability!


In [6]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s.%(msecs)03d %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)
logger.info(f"{__name__}: Hello")

09:43:30 INFO:__main__: Hello


In [48]:
# Hyperparameters
batch_size = 32
block_size = 8
n_embed = 32 ## Number of dimensions of the token embedding
max_iters = 3000
eval_interval = 300
eval_iters = 200
learning_rate = 1e-2
if IN_COLAB:
    device = 'cuda' if torch.cuda.is_available() else xm.xla_device() ### For TPU in Google collab
else:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

In [ ]:
input_file = "/content/gdrive/My Drive/data/tiny_shakespeare.txt" # "/content/tiny_shakespeare.txt"
with open(input_file, 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print(f"length of text = {len(text)}")
print(text[:100])

length of text = 1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [ ]:
# Unique Characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(','.join(chars))
print(f"Vocab Size = {vocab_size}")


, ,!,$,&,',,,-,.,3,:,;,?,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
Vocab Size = 65


In [ ]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}
encode = lambda s: [ stoi[c] for c in s] # encode a string to a list of integers
decode = lambda l: ''.join([ itos[i] for i in l]) # decode a list of integers back to a string

In [ ]:
# Tokenize the entire text and store into torch tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Shape: {data.shape}, Dtype: {data.dtype}")
print(data[:100]) ## Looks like 0 = \n, 1 = ' '

Shape: torch.Size([1115394]), Dtype: torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [ ]:
# Split in train, val
tv_split = int(0.9*len(data))
train_data = data[:tv_split]
val_data = data[tv_split:]
print(f"Split Index = {tv_split}, Training len = {len(train_data)}, Validation len = {len(val_data)}")

Split Index = 1003854, Training len = 1003854, Validation len = 111540


In [ ]:
def get_batch(split, batch_size): ## Batch Size = Number of sequences being processed in parallel!
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Generate `batch_size` random offsets
    x = torch.stack([data[i:i+block_size] for i in ix ]) # Each sample is stacked as a row!
    y = torch.stack([data[i+1:i+block_size+1] for i in ix ])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def print_estimate_loss(model, iter):
    xlosses = {}
    model.eval() ### Put in `inference` mode [Not needed in our case, no different behaviour for our model!]
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split, batch_size=batch_size)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        xlosses[split] = losses.mean()
        model.train()
    logger.info(f"step {iter}: train loss = {xlosses['train']}, val loss = {xlosses['val']}")
    return xlosses

In [ ]:
def generate_text(num_tokens=100):
    ## Create the initial 'text' to generate the continuation --> Using 0 = \n
    idx = torch.zeros((1,1), dtype=torch.long, device=device)
    tokens = m.generate(idx, num_tokens=num_tokens)
    print(decode(tokens[0].tolist()))

In [92]:
# Attention Head Module

class AttentionHead(nn.Module):
    """Single Attention head"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) ## T x T

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x) # B x T x head_size (hs)
        q = self.query(x) # B x T x head_size (hs)

        wei = q @ k.transpose(-2, -1) * C**-0.5 # Scaled attention
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        v = self.value(x) # B x T x head_size (hs)
        out = wei @ v
        return out        


In [94]:
class MultiHeadAttention(nn.Module):
    """Multiple Attention heads running in parallel"""
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for _ in range(num_heads)])

    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1) # Concatenate on the `channel` dimension

In [95]:
class FeedForward(nn.Module):
    """Positional Feed Forward"""
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, n_embed),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

In [89]:
# Transformer Language Model
class TransformerLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Each token gets the logits for the next token from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed)
        self.sa_head = AttentionHead(n_embed//2)
        # 1. self.sa_heads = MultiHeadAttention(4, n_embed//4) ## 4 heads of 8-dimensional self-attention
        # 2. self.ffwd = FeedForward(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx, targets=None):
        # idx, targets --> B x T (batch_size x block_size)
        token_embedding = self.token_embedding_table(idx) # B x T x C (n_embed)
        position_embedding = self.position_embedding_table(torch.arange(T, device=device)) # (T x C)
        x = token_embedding + position_embedding ## B x T x C (position_embedding gets broadcasted for each batch)
        x = self.sa_head(x) # Apply a single attention head
        # x = self.sa_heads(x) # Apply the attention heads
        # x = self.ffwd(x) # B x T x C
        logits = self.lm_head(x) # B x T x vocab_size

        if targets is None:
            loss = None
        else:
            # loss = F.cross_entropy(logits, targets) # Does not work because pytorch needs B * C * T for multi-dimensional array
            # So, Reshaping so that cross_entropy works
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, num_tokens): ## Generate next `num_tokens` tokens, idx --> B x T
        for _ in range(num_tokens):
            # crop idx to the last block size tokens [Remove extra tokens from the beginning!]
            # because positional encodings are defined only upto block_size
            idx_cond = idx[:, -block_size:]
            # get the predictions/losses
            logits, loss = self(idx_cond) ## logits --> B x T x C
            # focus on last time step (only the last character)
            logits = logits[:, -1, :] ## --> B x C
            # counts = logits.exp() # counts, equivalent to N
            # probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
            probs = F.softmax(logits, dim=-1) ## --> B x C
            # Sample from the probability distribution to get the next idx.
            idx_next = torch.multinomial(probs, num_samples=1) ## --> B x 1
            ## Question: should this not be idx_cond??
            idx = torch.cat((idx, idx_next), dim=1) ## --> B x T+1
        return idx


m = TransformerLanguageModel()
m.to(device)


TransformerLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (position_embedding_table): Embedding(8, 32)
  (sa_head): Head(
    (key): Linear(in_features=32, out_features=32, bias=False)
    (query): Linear(in_features=32, out_features=32, bias=False)
    (value): Linear(in_features=32, out_features=32, bias=False)
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

In [ ]:
# create an optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # instead of torch.optim.SGD

In [ ]:
for iter in range(max_iters): ## `max_iters` iterations

    if not iter % eval_interval:
        print_estimate_loss(m, iter)
    xb, yb = get_batch('train', batch_size=batch_size) ## xb = B x T
    # print(f"Shapes: {xb.shape} / {yb.shape}")
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) ## Zero out existing gradients computed for previous step
    loss.backward()
    optimizer.step() ## change the weights based on the gradients
    # print(loss.item())

print_estimate_loss(m, iter)

04:35:17 INFO:step 1: train loss = 4.725739479064941, val loss = 4.721153736114502
04:35:22 INFO:step 2: train loss = 4.72503137588501, val loss = 4.719532012939453
04:35:27 INFO:step 3: train loss = 4.717853546142578, val loss = 4.7250752449035645
04:35:32 INFO:step 4: train loss = 4.724914073944092, val loss = 4.719893932342529
04:35:37 INFO:step 5: train loss = 4.717930793762207, val loss = 4.724152088165283
04:35:42 INFO:step 6: train loss = 4.715250015258789, val loss = 4.714425563812256
04:35:48 INFO:step 7: train loss = 4.720458507537842, val loss = 4.715435981750488
04:35:54 INFO:step 8: train loss = 4.713040351867676, val loss = 4.715363025665283
04:36:00 INFO:step 9: train loss = 4.71272087097168, val loss = 4.711820602416992
04:36:07 INFO:step 10: train loss = 4.712316513061523, val loss = 4.719632148742676
04:36:13 INFO:step 11: train loss = 4.719141960144043, val loss = 4.715182304382324
04:36:19 INFO:step 12: train loss = 4.718740463256836, val loss = 4.711416721343994
04

In [ ]:
generate_text(num_tokens=300)

In [42]:
# Just a quick demo of tril to show how to get averages of previous rows/tokens
torch.manual_seed(42)   
a = torch.tril(torch.ones(3,3)) ## lower triangular portion (Previous tokens only!)
a = a / torch.sum(a, dim=1, keepdim=True) ## To average out the values of all previous tokens/rows!
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")


a = tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b = tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c = tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [77]:
# Mathematical trick in self attention
# Make the tokens talk to past token -- # token # 5 talks to 0,1,2,3,4

torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C) ### batch x time component/tokens x channels

# we want x[b, t] = mean[i <= t] in x[b, i]
# Method 1: Non Matrix Method.
xbow = torch.zeros(B,T,C) ## bow = bag of words!
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t, C
        xbow[b,t] = torch.mean(xprev, 0) # Averaging over dim=0 (time/tokens)

# Method 2: With Tril/Averaging
wei = torch.tril(torch.ones(T, T)) ## T x T
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x ## (T, T) x (B, T, C) ==> Because of broadcasting (T, T) => (B, T, T) ==> Output = B x T x C
print(f"Xbow2 allclose = {torch.allclose(xbow, xbow2)}")

# Method 3: Using softmax
tril = torch.tril(torch.ones(T, T)) ## T x T
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) ## Same as the wei matrix in Method 2
xbow3 = wei @ x
print(f"Xbow3 allclose = {torch.allclose(xbow, xbow3)}")

Xbow2 allclose = True
Xbow3 allclose = True


In [86]:
# Self Attention

torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C) ### batch x time component/tokens x channels

# Single Attention Head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B x T x head_size (hs)
q = query(x) # B x T x head_size (hs)
v = value(x) # B x T x head_size (hs)

## Transpose on the last 2 dimensions! (B x T x hs) @ (B x hs x T) ==> B x T x T
## Scale by the attention head size to ensure that the variance is kept low.
## If the variance is high, then softmax will be sharpened to the max and become like a one-hot vector.
wei = q @ k.transpose(-2, -1) * head_size**-0.5

print(f"Shapes/Variance: k = {k.shape} // {k.var()}, q = {q.shape} // {q.var()}, wei = {wei.shape} // {wei.var()}")

tril = torch.tril(torch.ones(T, T)) ## T x T
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1) # Apply softmax to the last dimension. (first row of each wei wil be 1,0,0....)
# Basically, now, wei is not averaged out across previous tokens, but is data dependent....

out = wei @ v
print(f"Out shape: {out.shape}, Variance of Weights {wei.var()}")

wei

Shapes/Variance: k = torch.Size([4, 8, 16]) // 0.3163500726222992, q = torch.Size([4, 8, 16]) // 0.33860543370246887, wei = torch.Size([4, 8, 8]) // 1.9223554134368896
Out shape: torch.Size([4, 8, 16]), Variance of Weights 0.04722575843334198


tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089